<a href="https://colab.research.google.com/github/seungminhama/portfolio/blob/main/projects/finance/regional_economy_map/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경설정 및 데이터 로드

In [ ]:
!pip install tqdm

In [ ]:
# 필요한 라이브러리 임포트
import os, re, glob, io, zipfile, warnings
import math, time, requests
import folium, json
import geopandas as gpd
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
from matplotlib.colors import ListedColormap
import statsmodels.api as sm
from google.colab import files
from contextlib import contextmanager
from branca.colormap import linear
from folium import FeatureGroup
from folium.plugins import MarkerCluster, MeasureControl
from shapely.geometry import Polygon
from tqdm.auto import tqdm

In [ ]:
# Google Drive 마운트
drive.mount('/content/drive')

# 경고 무시
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Google Drive 경로 설정
data_path = '/content/drive/MyDrive/2025 천안 데이터 공모전/데이터'

# 데이터 정제

## 기숙사

In [ ]:
path_dorm = "/content/drive/MyDrive/데이터/충남_대학교_기숙사_2022_2024.csv"

In [ ]:
dorm = pd.read_csv(path_dorm)
print(dorm.shape)
print(dorm.columns)

(289, 33)
Index(['기준연도', '학교종류', '설립구분', '지역', '상태', '학교', '재학생수(A)', '총 실수',
       '수용가능인원(B)', '실제 수용 인원', 'Unnamed: 10', 'Unnamed: 11',
       '기숙사수용률\n(C=B/Ax100)', '기숙사 지원자 수(D)', 'Unnamed: 14', 'Unnamed: 15',
       '입사 경쟁률\n(E=D/B)', '의무식\n여부', '구분', '건물명', '준공연도', '1인실', 'Unnamed: 22',
       'Unnamed: 23', '2인실', 'Unnamed: 25', 'Unnamed: 26', '3인실',
       'Unnamed: 28', 'Unnamed: 29', '4인실 이상', 'Unnamed: 31', 'Unnamed: 32'],
      dtype='object')


In [ ]:
# 필요한 칼럼만 남기기
dorm_slim = dorm[['기준연도','지역','학교','실제 수용 인원']].copy()

print(dorm_slim.head())
print(dorm_slim.shape)

     기준연도   지역      학교 실제 수용 인원
0     NaN  NaN     NaN     학위과정
1     NaN  NaN     NaN      내국인
2  2022.0   충남  건양대학교       210
3     NaN   충남     NaN      130
4     NaN   충남     NaN      340
(289, 4)


In [ ]:
# 실제 수용 인원에 숫자 아닌 값은 NaN 처리
dorm_slim['실제 수용 인원'] = pd.to_numeric(dorm_slim['실제 수용 인원'], errors='coerce')

# NaN 값 제거
dorm_clean = dorm_slim.dropna(subset=['실제 수용 인원']).copy()

print(dorm_clean.shape)
print(dorm_clean.head(100))

(274, 4)
       기준연도  지역        학교  실제 수용 인원
2    2022.0  충남    건양대학교      210.0
3       NaN  충남       NaN     130.0
4       NaN  충남       NaN     340.0
5       NaN  충남       NaN     237.0
6       NaN  충남       NaN     112.0
..      ...  ..       ...       ...
101     NaN  충남       NaN     181.0
102     NaN  충남  공주교육대학교      186.0
103     NaN  충남       NaN     249.0
104     NaN  충남       NaN     206.0
105     NaN  충남  국립공주대학교      778.0

[100 rows x 4 columns]


In [ ]:
# 기준연도, 학교 NaN 채우기
dorm_clean[['기준연도','학교']] = dorm_clean[['기준연도','학교']].ffill()
dorm_clean['기준연도'] = dorm_clean['기준연도'].astype('Int64')

print(dorm_clean.head(10))
print(dorm_clean.tail(10))

    기준연도  지역        학교  실제 수용 인원
2   2022  충남    건양대학교      210.0
3   2022  충남    건양대학교      130.0
4   2022  충남    건양대학교      340.0
5   2022  충남    건양대학교      237.0
6   2022  충남    건양대학교      112.0
7   2022  충남    건양대학교        0.0
8   2022  충남  공주교육대학교      249.0
9   2022  충남  공주교육대학교      197.0
10  2022  충남  공주교육대학교      208.0
11  2022  충남  국립공주대학교      156.0
     기준연도  지역          학교  실제 수용 인원
278  2024  충남  한국전통문화대학교       36.0
279  2024  충남  한국전통문화대학교       98.0
280  2024  충남  한국전통문화대학교      111.0
281  2024  충남      한서대학교      963.0
282  2024  충남      한서대학교      335.0
283  2024  충남      한서대학교      497.0
284  2024  충남      호서대학교      796.0
286  2024  충남      호서대학교      458.0
287  2024  충남      호서대학교       31.0
288  2024  충남      호서대학교       77.0


In [ ]:
# 천안 소재 대학 리스트
cheonan_univ = [
    "단국대학교 _제2캠퍼스","상명대학교 _제2캠퍼스",
    "남서울대학교","백석대학교","호서대학교",
    "한국기술교육대학교","나사렛대학교"
]

# 학교명에 해당 대학명이 포함된 행만 남기기
mask = dorm_clean['학교'].astype(str).str.contains("|".join(cheonan_univ), regex=True)
dorm_cheonan = dorm_clean[mask].copy()

print(dorm_cheonan['학교'].unique())
print(dorm_cheonan.head())

['나사렛대학교 ' '남서울대학교 ' '단국대학교 _제2캠퍼스' '백석대학교 ' '상명대학교 _제2캠퍼스' '한국기술교육대학교 '
 '호서대학교 ']
    기준연도  지역       학교  실제 수용 인원
24  2022  충남  나사렛대학교      292.0
25  2022  충남  나사렛대학교      196.0
26  2022  충남  나사렛대학교      358.0
27  2022  충남  남서울대학교       12.0
28  2022  충남  남서울대학교       27.0


## 천안시 인구

In [ ]:
path_pop = "/content/drive/MyDrive/데이터/연령_및_성별_인구_–_읍면동_20250830034743.csv"

In [ ]:
pop = pd.read_csv(path_pop, encoding='cp949')
print(pop.shape)
print(pop.columns)
print(pop.head(20))

(7, 12)
Index(['행정구역별(읍면동)', '연령별', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', '2024'],
      dtype='object')
   행정구역별(읍면동)     연령별    2015    2016    2017    2018    2019    2020    2021  \
0  행정구역별(읍면동)     연령별  내국인(명)  내국인(명)  내국인(명)  내국인(명)  내국인(명)  내국인(명)  내국인(명)   
1         천안시  20~24세   50197   51362   51889   51765   51911   49777   48599   
2         천안시  25~29세   42390   42341   45341   46940   47784   50008   50124   
3         동남구  20~24세   24130   24150   23950   23962   24377   22110   21065   
4         동남구  25~29세   16520   16289   16781   17253   17633   17851   17616   
5         서북구  20~24세   26067   27212   27939   27803   27534   27667   27534   
6         서북구  25~29세   25870   26052   28560   29687   30151   32157   32508   

     2022    2023    2024  
0  내국인(명)  내국인(명)  내국인(명)  
1   48263   45748   44987  
2   48870   47742   46922  
3   21586   20698   20476  
4   17411   17333   17080  
5   26677   25050   24511  
6   31459

In [ ]:
# 전체인구
path_pop_all = "/content/drive/MyDrive/데이터/연령_및_성별_인구_–_읍면동_20250828212700_전국.csv"

pop_all = pd.read_csv(path_pop_all, encoding="cp949", dtype=str, low_memory=False)

print(pop_all.head())

   행정구역별(읍면동)     연령별     2015     2016     2017     2018     2019     2020  \
0  행정구역별(읍면동)     연령별   내국인(명)   내국인(명)   내국인(명)   내국인(명)   내국인(명)   내국인(명)   
1          전국  20~24세  3385936  3400634  3355986  3290233  3203741  3193316   
2          전국  25~29세  3027896  3068970  3164039  3259284  3343423  3423231   

      2021     2022     2023     2024  
0   내국인(명)   내국인(명)   내국인(명)   내국인(명)  
1  3066118  2879013  2715480  2579629  
2  3439492  3384426  3310331  3214861  


## 이동인구

In [ ]:
path_move_2015 = "/content/drive/MyDrive/데이터/2015_인구관련연간자료_20250825_12667.csv"
path_move_2016 = "/content/drive/MyDrive/데이터/2016_인구관련연간자료_20250825_12667.csv"
path_move_2017 = "/content/drive/MyDrive/데이터/2017_인구관련연간자료_20250825_12667.csv"
path_move_2018 = "/content/drive/MyDrive/데이터/2018_인구관련연간자료_20250825_12667.csv"
path_move_2019 = "/content/drive/MyDrive/데이터/2019_인구관련연간자료_20250825_12667.csv"
path_move_2020 = "/content/drive/MyDrive/데이터/2020_인구관련연간자료_20250825_60311.csv"
path_move_2021 = "/content/drive/MyDrive/데이터/2021_인구관련연간자료_20250825_60311.csv"
path_move_2022 = "/content/drive/MyDrive/데이터/2022_인구관련연간자료_20250825_60311.csv"
path_move_2023 = "/content/drive/MyDrive/데이터/2023_인구관련연간자료_20250825_60311.csv"
path_move_2024 = "/content/drive/MyDrive/데이터/2024_인구관련연간자료_20250825_60311.csv"

In [ ]:
move_2015 = pd.read_csv(path_move_2015, header=None, dtype=str)
move_2016 = pd.read_csv(path_move_2016, header=None, dtype=str)
move_2017 = pd.read_csv(path_move_2017, header=None, dtype=str)
move_2018 = pd.read_csv(path_move_2018, header=None, dtype=str)
move_2019 = pd.read_csv(path_move_2019, header=None, dtype=str)
move_2020 = pd.read_csv(path_move_2020, header=None, dtype=str)
move_2021 = pd.read_csv(path_move_2021, header=None, dtype=str)
move_2022 = pd.read_csv(path_move_2022, header=None, dtype=str)
move_2023 = pd.read_csv(path_move_2023, header=None, dtype=str)
move_2024 = pd.read_csv(path_move_2024, header=None, dtype=str)

In [ ]:
# 뒤쪽 완전 빈 칼림이 있으면 제거
move_2015 = move_2015.dropna(axis=1, how='all')

In [ ]:
# 칼럼 수에 맞춰 이름 부여
if move_2015.shape[1] < 14:
  # 부족한 만큼 NaN열 추가
  for i in range(14 - move_2015.shape[1]):
    move_2015[move_2015.shape[1]] = pd.NA
elif move_2015.shape[1] > 14:
  move_2015 = move_2015.iloc[:, :14]

# 칼럼 이름 고정
names14 = [
    '전입시도코드','전입시군구코드','전입연도','전입월',
    '전출시도코드','전출시군구코드',
    '전입자1_만연령','전입자1_성별코드',
    '전입자2_만연령','전입자2_성별코드',
    '전입자3_만연령','전입자3_성별코드',
    '전입자4_만연령', '전입자4_성별코드'
]
move_2015.columns = names14

# 코드형 변수 zero-padding 처리
for c, width in [('전입시도코드',2), ('전출시도코드',2)]:
    move_2015[c] = move_2015[c].astype(str).str.replace('.0','', regex=False).str.zfill(width)
for c, width in [('전입시군구코드',3), ('전출시군구코드',3)]:
    move_2015[c] = move_2015[c].astype(str).str.replace('.0','', regex=False).str.zfill(width)\

# 숫자형으로 바꿀 수 있는 칼럼 변환 (연도, 월, 연령, 성별)
num_cols = [c for c in move_2015.columns if '연도' in c or '월' in c or '만연령' in c or '성별코드' in c]
for c in num_cols:
    move_2015[c] = pd.to_numeric(move_2015[c], errors='coerce')

print(move_2015.head())

  전입시도코드 전입시군구코드  전입연도  전입월 전출시도코드 전출시군구코드  전입자1_만연령  전입자1_성별코드  전입자2_만연령  \
0     11     110  2015    1     11     110       2.0        4.0       1.0   
1     11     110  2015    1     11     110       6.0        3.0       NaN   
2     11     110  2015    1     11     110       8.0        3.0       NaN   
3     11     110  2015    1     11     110      10.0        4.0       8.0   
4     11     110  2015    1     11     110      12.0        4.0       NaN   

   전입자2_성별코드  전입자3_만연령  전입자3_성별코드  전입자4_만연령  전입자4_성별코드  
0        3.0       NaN        NaN       NaN        NaN  
1        NaN       NaN        NaN       NaN        NaN  
2        NaN       NaN        NaN       NaN        NaN  
3        3.0       NaN        NaN       NaN        NaN  
4        NaN       NaN        NaN       NaN        NaN  


In [ ]:
# 정수로 보고 싶은 컬럼들
int_cols = [
    '전입시도코드','전입시군구코드','전입연도','전입월',
    '전출시도코드','전출시군구코드',
    '전입자1_만연령','전입자1_성별코드',
    '전입자2_만연령','전입자2_성별코드',
    '전입자3_만연령','전입자3_성별코드',
    '전입자4_만연령', '전입자4_성별코드'
]

for c in int_cols:
    move_2015[c] = pd.to_numeric(move_2015[c], errors='coerce').astype('Int64')
    # <- pandas의 Nullable Int (결측 NaN도 표현 가능)

print(move_2015.head())

   전입시도코드  전입시군구코드  전입연도  전입월  전출시도코드  전출시군구코드  전입자1_만연령  전입자1_성별코드  전입자2_만연령  \
0      11      110  2015    1      11      110         2          4         1   
1      11      110  2015    1      11      110         6          3      <NA>   
2      11      110  2015    1      11      110         8          3      <NA>   
3      11      110  2015    1      11      110        10          4         8   
4      11      110  2015    1      11      110        12          4      <NA>   

   전입자2_성별코드  전입자3_만연령  전입자3_성별코드  전입자4_만연령  전입자4_성별코드  
0          3      <NA>       <NA>      <NA>       <NA>  
1       <NA>      <NA>       <NA>      <NA>       <NA>  
2       <NA>      <NA>       <NA>      <NA>       <NA>  
3          3      <NA>       <NA>      <NA>       <NA>  
4       <NA>      <NA>       <NA>      <NA>       <NA>  


In [ ]:
age_cols = [c for c in move_2015.columns if '만연령' in c]
print(age_cols)

['전입자1_만연령', '전입자2_만연령', '전입자3_만연령', '전입자4_만연령']


In [ ]:
for col in age_cols:
    s = (move_2015[col]
            .astype(str)                      # 문자열화(혼합형 대비)
            .str.strip()
            .str.replace(',', '', regex=False)
            .str.replace('.0', '', regex=False))
    s = pd.to_numeric(s, errors='coerce')     # ← 숫자로 강제 변환
    s = s.where(s >= 10, s * 10)              # ← 한 자리면 ×10 보정
    move_2015[col] = s.astype('Int64')        # ← .0 제거(결측 허용 정수)

# 보정 결과 확인
print(move_2015[age_cols].head(10))

   전입자1_만연령  전입자2_만연령  전입자3_만연령  전입자4_만연령
0        20        10      <NA>      <NA>
1        60      <NA>      <NA>      <NA>
2        80      <NA>      <NA>      <NA>
3        10        80      <NA>      <NA>
4        12      <NA>      <NA>      <NA>
5        15        80        39      <NA>
6        16        11        39      <NA>
7        19      <NA>      <NA>      <NA>
8        20      <NA>      <NA>      <NA>
9        20      <NA>      <NA>      <NA>


In [ ]:
mask = False
for col in age_cols:
    mask |= move_2015[col].between(20, 29, inclusive="both")

move_2015_youth = move_2015[mask].copy()
print(move_2015_youth[age_cols].head())

    전입자1_만연령  전입자2_만연령  전입자3_만연령  전입자4_만연령
0         20        10      <NA>      <NA>
8         20      <NA>      <NA>      <NA>
9         20      <NA>      <NA>      <NA>
10        20      <NA>      <NA>      <NA>
11        20      <NA>      <NA>      <NA>


In [ ]:
cols_to_show = [
    '전입시도코드','전입시군구코드','전입연도','전입월',
    '전출시도코드','전출시군구코드'
] + age_cols

print(move_2015_youth[cols_to_show].head(20))

    전입시도코드  전입시군구코드  전입연도  전입월  전출시도코드  전출시군구코드  전입자1_만연령  전입자2_만연령  전입자3_만연령  \
0       11      110  2015    1      11      110        20        10      <NA>   
8       11      110  2015    1      11      110        20      <NA>      <NA>   
9       11      110  2015    1      11      110        20      <NA>      <NA>   
10      11      110  2015    1      11      110        20      <NA>      <NA>   
11      11      110  2015    1      11      110        20      <NA>      <NA>   
12      11      110  2015    1      11      110        21      <NA>      <NA>   
13      11      110  2015    1      11      110        21      <NA>      <NA>   
14      11      110  2015    1      11      110        21      <NA>      <NA>   
15      11      110  2015    1      11      110        21      <NA>      <NA>   
16      11      110  2015    1      11      110        21      <NA>      <NA>   
17      11      110  2015    1      11      110        21      <NA>      <NA>   
18      11      110  2015   

In [ ]:
# 전입자3, 4 관련 칼럼
cols_3_4 = [
    '전입자3_만연령','전입자3_성별코드',
    '전입자4_만연령','전입자4_성별코드'
]

# 전입자3, 4의 값이 NaN만 남기기
move_2015_youth_nan = move_2015_youth[move_2015_youth[cols_3_4].isna().all(axis=1)].copy()

# 전입자3, 4 열 지우기
move_2015_drop = move_2015_youth_nan.drop(columns=cols_3_4)

print(move_2015_drop.head())

    전입시도코드  전입시군구코드  전입연도  전입월  전출시도코드  전출시군구코드  전입자1_만연령  전입자1_성별코드  \
0       11      110  2015    1      11      110        20          4   
8       11      110  2015    1      11      110        20          2   
9       11      110  2015    1      11      110        20          2   
10      11      110  2015    1      11      110        20          2   
11      11      110  2015    1      11      110        20          2   

    전입자2_만연령  전입자2_성별코드  
0         10          3  
8       <NA>       <NA>  
9       <NA>       <NA>  
10      <NA>       <NA>  
11      <NA>       <NA>  


In [ ]:
# 천안 전입자
df = move_2015_drop.reset_index(drop=True)  # 인덱스 정렬/초기화

# 코드 비교는 문자열로 안전하게 (44 / 131 / 133)
mask_in = (df['전입시도코드'].astype(str) == '44') & (
    df['전입시군구코드'].astype(str).isin(['131','133'])
)

mask_out = (df['전출시도코드'].astype(str) == '44') & (
    df['전출시군구코드'].astype(str).isin(['131','133'])
)

move_in_cheonan  = df.loc[mask_in].copy()
move_out_cheonan = df.loc[mask_out].copy()
move_cheonan     = df.loc[mask_in | mask_out].copy()

print(move_cheonan.head())

     전입시도코드  전입시군구코드  전입연도  전입월  전출시도코드  전출시군구코드  전입자1_만연령  전입자1_성별코드  \
373      11      110  2015    1      44      131        22          2   
374      11      110  2015    1      44      131        27          2   
375      11      110  2015    1      44      133        23          1   
376      11      110  2015    1      44      133        24          2   
854      11      110  2015    2      44      131        24          2   

     전입자2_만연령  전입자2_성별코드  
373      <NA>       <NA>  
374      <NA>       <NA>  
375      <NA>       <NA>  
376      <NA>       <NA>  
854      <NA>       <NA>  


In [ ]:
NAMES14 = [
    '전입시도코드','전입시군구코드','전입연도','전입월',
    '전출시도코드','전출시군구코드',
    '전입자1_만연령','전입자1_성별코드',
    '전입자2_만연령','전입자2_성별코드',
    '전입자3_만연령','전입자3_성별코드',
    '전입자4_만연령','전입자4_성별코드'
]
AGE_COLS = ['전입자1_만연령','전입자2_만연령','전입자3_만연령','전입자4_만연령']
COLS_3_4 = ['전입자3_만연령','전입자3_성별코드','전입자4_만연령','전입자4_성별코드']
CHEONAN_GU = {'131','133'}

def _zero_pad_series(s, width):
    # 문자열화 최소화: NaN은 빈문자 처리 후 제로패딩
    s = s.astype(object)
    s = s.where(~s.isna(), '')              # NaN -> ''
    s = s.astype(str).str.replace(r'\.0$', '', regex=True)
    return s.str.zfill(width)

def preprocess_move_df(df: pd.DataFrame) -> pd.DataFrame:
    # 0) 사본 한 번만
    df = df.copy()

    # 1) 항상 14개 컬럼으로 맞추기 (앞 14개 사용 + 부족분 NA 채움)
    if df.shape[1] >= 14:
        df = df.iloc[:, :14]
    else:
        df = pd.concat([df, pd.DataFrame(np.nan, index=df.index, columns=range(14 - df.shape[1]))], axis=1)
    df.columns = NAMES14

    # 2) 코드 제로패딩 (문자열 유지)
    df['전입시도코드']  = _zero_pad_series(df['전입시도코드'], 2)
    df['전출시도코드']  = _zero_pad_series(df['전출시도코드'], 2)
    df['전입시군구코드'] = _zero_pad_series(df['전입시군구코드'], 3)
    df['전출시군구코드'] = _zero_pad_series(df['전출시군구코드'], 3)

    # 3) 숫자형 변환 (연도/월/연령/성별만)
    num_cols = [c for c in NAMES14 if ('연도' in c or '월' in c or '만연령' in c or '성별코드' in c)]
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

    # 4) 나이 보정: 한 자리(예: 2 → 20)는 *10, 나머지는 그대로
    ages = df[AGE_COLS].to_numpy(dtype='float64', copy=True)
    small_mask = (ages >= 0) & (ages < 10)
    ages[small_mask] *= 10
    df[AGE_COLS] = ages

    # 5) 20~29세 전입자 존재 행 필터 (배열 연산)
    age_arr = df[AGE_COLS].to_numpy(dtype='float64', copy=False)
    has_20s = np.any((age_arr >= 20) & (age_arr <= 29), axis=1)
    df = df.loc[has_20s].copy()

    # 6) 전입자3/4 값 있는 행 제외 후 해당 열 삭제
    if set(COLS_3_4).issubset(df.columns):
        only_na_34 = df[COLS_3_4].isna().all(axis=1)
        df = df.loc[only_na_34].drop(columns=COLS_3_4)

    # 7) 천안(충남-동남/서북) 관련 행만
    mask_in  = (df['전입시도코드'] == '44') & (df['전입시군구코드'].isin(CHEONAN_GU))
    mask_out = (df['전출시도코드'] == '44') & (df['전출시군구코드'].isin(CHEONAN_GU))
    df = df.loc[mask_in | mask_out].reset_index(drop=True)

    # 8) 경량 다운캐스트 (메모리 절약)
    for c in ['전입연도','전입월']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='integer')
    for c in ['전입자1_성별코드','전입자2_성별코드']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='integer')
    for c in ['전입자1_만연령','전입자2_만연령']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], downcast='integer')

    return df

In [ ]:
# 연도별 원본 DataFrame 딕셔너리 (이미 불러온 상태)
raw_dfs = {
    2015: move_2015, 2016: move_2016, 2017: move_2017, 2018: move_2018, 2019: move_2019,
    2020: move_2020, 2021: move_2021, 2022: move_2022, 2023: move_2023, 2024: move_2024
}

# 전처리 함수 적용
processed = [preprocess_move_df(df) for df in raw_dfs.values()]

# 합치기
all_moves = pd.concat(processed, ignore_index=True)

print(all_moves.shape)
print(all_moves.head())

(373737, 10)
  전입시도코드 전입시군구코드  전입연도  전입월 전출시도코드 전출시군구코드  전입자1_만연령  전입자1_성별코드  전입자2_만연령  \
0     11     110  2015    1     44     131        22          2       NaN   
1     11     110  2015    1     44     131        27          2       NaN   
2     11     110  2015    1     44     133        23          1       NaN   
3     11     110  2015    1     44     133        24          2       NaN   
4     11     110  2015    2     44     131        24          2       NaN   

   전입자2_성별코드  
0       <NA>  
1       <NA>  
2       <NA>  
3       <NA>  
4       <NA>  


In [ ]:
processed = []
for year, df in raw_dfs.items():
    tmp = preprocess_move_df(df)
    tmp['파일연도'] = year   # 파일 이름상 연도
    processed.append(tmp)

all_moves = pd.concat(processed, ignore_index=True)

## 소비

In [ ]:
path_consume = "/content/drive/MyDrive/데이터/소상공인시장진흥공단_충남_데이터정리.csv"

In [ ]:
consume = pd.read_csv(path_consume, encoding="cp949")

print(consume.shape)
print(consume.columns)
print(consume.head(20))

(121049, 39)
Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')
                  상가업소번호        상호명  지점명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드  \
0   MA010120220806289059      도미당약국  NaN        G2       소매      G215   
1   MA010120220806289069     꽃지바다횟집  NaN        I2       음식      I201   
2   MA010120220806288745    신바람할인매장  NaN        G2       소매      G222   
3   MA010120220806290247      성도카센타  NaN        S2    수리·개인      S203   
4   MA010120220806289855        A민박  NaN        I1       숙박      I101   
5   MA010120220806290207       한진식당  NaN        I2       음식      I201   
6   MA010120220806291625        울돌목  NaN        I2       음식

In [ ]:
# 천안시만 필터링
# 필터 — 코드가 정수형이면 이렇게
consume_cheonan = consume[
    (consume['시도코드'] == 44) &
    (consume['시군구코드'].isin([44131, 44133]))
].copy()

# 코드가 문자열일 수도 있으니 안전하게 하려면:
# consume_cheonan = consume[
#     (consume['시도코드'].astype(str).str.strip() == '44') &
#     (consume['시군구코드'].astype(str).str.strip().isin(['44131','44133']))
# ].copy()

# (2) 전체 행/열 확인
print(consume_cheonan.shape)      # 전체 행 수
print(consume_cheonan.head(10))   # 앞 10행 (중복 제거 금지!)

# (3) 분포 확인(참고)
print(consume_cheonan['시군구코드'].value_counts())
print(consume_cheonan['시군구명'].value_counts())

(35865, 39)
                   상가업소번호        상호명  지점명 상권업종대분류코드 상권업종대분류명 상권업종중분류코드  \
9    MA010120220809678536       삼은기업  NaN        N1  시설관리·임대      N101   
19   MA010120220808510981        갯마을  NaN        I2       음식      I211   
102  MA010120220805106825      컴퓨터수리  NaN        S2    수리·개인      S201   
128  MA010120220805107025  담은푸드고기특별시  NaN        I2       음식      I201   
186  MA010120220807383959      라보우살롱  NaN        S2    수리·개인      S207   
189  MA010120220808511961  라이프아트협동조합  NaN        N1  시설관리·임대      N108   
228  MA010120220809691183    스튜디오비비드  NaN        M1    과학·기술      M113   
231  MA010120220800142391        장어왕  NaN        I2       음식      I201   
239  MA010120220807388786     개미인력개발  NaN        N1  시설관리·임대      N104   
249  MA010120220807385544       데이가구  NaN        G2       소매      G211   

      상권업종중분류명 상권업종소분류코드           상권업종소분류명 표준산업분류코드  ...        건물관리번호  \
9         시설관리    N10101    사업시설 유지·관리 서비스업   N74100  ...  4.413330e+24   
19          주점    I2110

## 이동수단

In [ ]:
from math import radians, cos, sin, asin, sqrt
import requests, time
from urllib.parse import urlencode

In [ ]:
path_under = "/content/drive/MyDrive/데이터/충청남도 천안시_지하차도 현황_20240728.csv"
path_bus = "/content/drive/MyDrive/데이터/충청남도 천안시_시내버스운수업체별노선현황_20240719.csv"
path_ped = "/content/drive/MyDrive/데이터/충청남도 천안시_보행자전용도로 현황_20230828.csv"
path_road = "/content/drive/MyDrive/데이터/충청남도 천안시_도로 현황_20230828.csv"
path_facilities = "/content/drive/MyDrive/데이터/충청남도 천안시_교통시설 현황_20230828.csv"

In [ ]:
under = pd.read_csv(path_under, encoding="cp949")
bus = pd.read_csv(path_bus, encoding="cp949")
ped = pd.read_csv(path_ped, encoding="cp949")
road = pd.read_csv(path_road, encoding="cp949")
facilities = pd.read_csv(path_facilities, encoding="cp949")

In [ ]:
print(under.shape)
print(under.columns)
print(under.head())

(24, 11)
Index(['연번', '시설물명', '주소', '종별', '시설물종류', '준공연도', '연장(미터)', '폭(미터)', '차선',
       '높이(미터)', '비고(경과년수)'],
      dtype='object')
   연번    시설물명                             주소  종별 시설물종류  준공연도  연장(미터)  폭(미터)  \
0   1   쌍용지하도     충청남도 천안시 서북구 쌍용동 1178번지 일원  3종   지하도  1990   219.0   18.1   
1   2  성정지하차도  충청남도 천안시 서북구 성정동 609-199번지 일원  3종   지하도  1983    72.4    6.1   
2   3  구상골지하도      충청남도 천안시 서북구 성정동 795번지 일원  3종   지하도  1993    25.0    6.0   
3   4  미라골지하도      충청남도 천안시 서북구 쌍용동 998번지 일원  3종   지하도  1993    30.0    6.0   
4   5   백석지하도      충청남도 천안시 서북구 성정동 940번지 일원  3종   지하도  1996    21.4    6.0   

    차선  높이(미터)  비고(경과년수)  
0  4.0     4.7        34  
1  NaN     2.5        41  
2  NaN     3.0        31  
3  NaN     3.0        31  
4  NaN     3.0        28  


In [ ]:
print(bus.shape)
print(bus.columns)
print(bus.head())

(149, 7)
Index(['연번', '노선번호', '운수업체명', '1회운행거리(km_편도기준)', '1일운행횟수(회_편도기준)', '비고',
       '데이터기준일자'],
      dtype='object')
   연번  노선번호      운수업체명  1회운행거리(km_편도기준)  1일운행횟수(회_편도기준)   비고     데이터기준일자
0   1     1  보성,삼안,새천안             21.3             103  NaN  2024-07-19
1   2     2  보성,삼안,새천안             23.3              55   편도  2024-07-19
2   3     3  보성,삼안,새천안             24.9              42   편도  2024-07-19
3   4     5  보성,삼안,새천안             21.3              96  NaN  2024-07-19
4   5     6  보성,삼안,새천안             22.0              47   편도  2024-07-19


In [ ]:
print(ped.shape)
print(ped.columns)
print(ped.head())

(9, 4)
Index(['현황도형 관리번호', '라벨명', '면적_도형', '길이_도형'], dtype='object')
                  현황도형 관리번호      라벨명        면적_도형        길이_도형
0  44130UQ161PS199604200001  보행자전용도로  2496.448840   929.293127
1  44130UQ161PS200511300001  보행자전용도로  8115.005114  4804.421495
2  44130UQ161PS200812010055  보행자전용도로   577.678693   584.225388
3  44130UQ161PS201302120001  보행자전용도로  1017.296160  1018.008044
4  44130UQ161PS201302210004  보행자전용도로  1976.623350   807.090973


In [ ]:
print(road.shape)
print(road.columns)
print(road.head())

(3360, 4)
Index(['현황도형 관리번호', '라벨명', '면적_도형', '길이_도형'], dtype='object')
                  현황도형 관리번호   라벨명         면적_도형        길이_도형
0  44130UQ151PS198706120134  중로2류  16010.823170  2150.773347
1  44130UQ151PS199005090004  중로2류   1817.963367   322.501439
2  44130UQ151PS199303060878  중로2류   2504.330546   381.979392
3  44130UQ151PS200101180426  중로2류   6633.100789   915.902343
4  44130UQ151PS200812011162  중로2류   1824.568938   257.065713


In [ ]:
print(facilities.shape)
print(facilities.columns)
print(facilities.head())

(155, 4)
Index(['현황도형 관리번호', '라벨명', '면적_도형', '길이_도형'], dtype='object')
                  현황도형 관리번호    라벨명        면적_도형       길이_도형
0  44130UQ152PS199711200007  노외주차장  1500.042180  155.955516
1  44130UQ152PS200002120002  노외주차장  1490.893627  169.014550
2  44130UQ152PS200002120003  노외주차장  1754.043494  185.787167
3  44130UQ152PS200002120004  노외주차장   824.525696  112.160168
4  44130UQ152PS200002120001  노외주차장  1467.094056  164.608955


In [ ]:
APIKEY   = "20250828161411g36ukorpo9ruqsavqljuj33mdg"   # 공공키
BASE_STOP  = "https://stcis.go.kr/openapi/bussttn.json"        # 정류장
BASE_ROUTE = "https://stcis.go.kr/openapi/busroute.json"       # 노선

# 행정코드: 충남/천안(동남/서북)
SD_CODE   = "44"                       # 충청남도
CITY_CODES = ["44131", "44133"]        # 천안 동남구, 서북구

# (선택) OD 기간 설정 — 필요에 맞게 바꾸세요 (YYYYMMDD 또는 문서 형식)
OD_START = "20240701"
OD_END   = "20240731"

In [ ]:
# ===== 공통 유틸 =====
def call_api(base_url: str, params: dict, timeout=30, retry=3, sleep=0.4):
    # API 호출 공통 래퍼 (JSON 응답 가정, 실패 시 재시도)
    p = dict(params)
    p["apikey"] = APIKEY
    last = None
    for i in range(retry):
        try:
            r = requests.get(base_url, params=p, timeout=timeout)
            r.raise_for_status()
            return r.json()
        except Exception as e:
            last = e
            time.sleep(sleep * (i+1))
    raise last

def parse_items(js):
    # 응답 JSON에서 리스트 꺼내기 — 흔한 키들을 자동 탐색
    if js is None:
        return []
    if isinstance(js, list):
        return js
    if isinstance(js, dict):
        for k in ["list", "items", "data", "result", "body", "rows"]:
            if k in js:
                v = js[k]
                if isinstance(v, dict) and "item" in v:
                    v = v["item"]
                if isinstance(v, list):
                    return v
                if isinstance(v, dict):
                    return [v]
        return [js]
    return []

def std_numeric(s):
    return (s.astype(str)
             .str.replace(",", "", regex=False)
             .str.extract(r"([-+]?\d*\.?\d+)", expand=False)
             .astype(float))

def soft_rename(df, mapping):
    # mapping에 존재하는 컬럼만 rename
    cols = {k:v for k,v in mapping.items() if k in df.columns}
    return df.rename(columns=cols)

In [ ]:
# ===== 수집 함수 =====
def fetch_stops(sdCd, sggCd):
    # 문서 기준: sdCd(시도), sggCd(시군구). 읍면동/ARS는 전체 수집이면 생략.
    js = call_api(BASE_STOP, {"sdCd": sdCd, "sggCd": sggCd})
    items = parse_items(js)
    df = pd.json_normalize(items)
    # 표준화 (응답 키 다양성 대비 후보를 넓게 잡음)
    df = soft_rename(df, {
        "sttnId":"stop_id", "sttnid":"stop_id", "정류장ID":"stop_id", "stopId":"stop_id",
        "sttnNm":"stop_name","정류장명":"stop_name","stopName":"stop_name","stationNm":"stop_name",
        "sttnArsno":"ars_id","arsId":"ars_id","ARS-ID":"ars_id",
        "lat":"lat","latitude":"lat","gpsLati":"lat","위도":"lat","y":"lat",
        "lon":"lon","longitude":"lon","gpsLong":"lon","경도":"lon","x":"lon",
        "sdCd":"sdCd","sggCd":"sggCd","emdCd":"emdCd"
    })
    # 숫자화(좌표가 온다면)
    for c in ["lat","lon"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    # 소속 구 코드 보강
    if "sggCd" not in df.columns:
        df["sggCd"] = sggCd
    return df

def fetch_routes(sdCd, sggCd):
    js = call_api(BASE_ROUTE, {"sdCd": sdCd, "sggCd": sggCd})
    items = parse_items(js)
    df = pd.json_normalize(items)
    df = soft_rename(df, {
        "routeId":"route_id","노선ID":"route_id","routeid":"route_id","ROUTE_ID":"route_id",
        "routeNo":"route_no","노선번호":"route_no","노선명":"route_no","ROUTE_NO":"route_no",
        "sdCd":"sdCd","sggCd":"sggCd"
    })
    if "sggCd" not in df.columns:
        df["sggCd"] = sggCd
    return df

In [ ]:
# ===== 실행: 수집 & 저장 =====
# 정류장
stops_list = []
for g in CITY_CODES:
    df = fetch_stops(SD_CODE, g)
    stops_list.append(df)
    time.sleep(0.2)

stops = pd.concat(stops_list, ignore_index=True).drop_duplicates()
# stop_id/ars_id 기준 중복제거 우선
key_cols = [c for c in ["stop_id","ars_id"] if c in stops.columns]
if key_cols:
    stops = stops.drop_duplicates(subset=key_cols)

# 노선
routes_list = []
for g in CITY_CODES:
    df = fetch_routes(SD_CODE, g)
    routes_list.append(df)
    time.sleep(0.2)
routes = pd.concat(routes_list, ignore_index=True).drop_duplicates()
if "route_id" in routes.columns:
    routes = routes.drop_duplicates(subset=["route_id"])

In [ ]:
# 간단 점검
print("▶ 정류장 정보")
print("shape:", stops.shape)
print("columns:", list(stops.columns))
print(stops.head(10))

▶ 정류장 정보
shape: (1646, 7)
columns: ['stop_id', 'bimsId', 'stop_name', 'ars_id', 'sdCd', 'sggCd', 'emdCd']
   stop_id     bimsId  stop_name ars_id sdCd  sggCd       emdCd
0  2920047  285000187       학계마을    187   44  44131  4413131026
1  2921027  288000931       세교1리    931   44  44131  4413131025
2  2921086  285000706    대림한숲아파트    706   44  44131  4413111800
3  2921087  285000707    대림한숲아파트    707   44  44131  4413111800
4  2921091  285002264    선영새마을금고   2264   44  44131  4413111400
5  2921092  285002351  일봉동행정복지센터   2351   44  44131  4413111400
6  2921097  285000709    대림한내아파트    709   44  44131  4413111800
7  2921098  285000708    대림한내아파트    708   44  44131  4413111800
8  2921099  285000901    공작아파트입구    901   44  44131  4413111300
9  2921109  285000898      순천향병원    898   44  44131  4413111300


## 편의/문화시설

In [ ]:
path_admin = "/content/drive/MyDrive/데이터/충청남도 천안시_행정기관정보_20230818.csv"
path_shelter = "/content/drive/MyDrive/데이터/충청남도 천안시_야외 휴게쉼터(정자 등)_20240710.csv"
path_outdoor = "/content/drive/MyDrive/데이터/충청남도 천안시_실외운동기구 현황_20240516.csv"
path_facility = "/content/drive/MyDrive/데이터/천안도시공사_시설현황_20250701.csv"

In [ ]:
admin = pd.read_csv(path_admin, encoding="cp949")
shelter = pd.read_csv(path_shelter, encoding="cp949")
outdoor = pd.read_csv(path_outdoor, encoding="cp949")
facility = pd.read_csv(path_facility, encoding="utf-8")

In [ ]:
print(admin.shape)
print(admin.columns)
print(admin.head())

(42, 5)
Index(['연번', '기관명', '소재지도로명주소', '전화번호', '데이터기준일자'], dtype='object')
   연번               기관명                    소재지도로명주소          전화번호     데이터기준일자
0   1         충청남도 천안시청        충청남도 천안시 서북구 번영로 156       1422-36  2023-08-18
1   2      충청남도 서북구 보건소        충청남도 천안시 서북구 번영로 156  041-521-2552  2023-08-18
2   3      충청남도 동남구 보건소         충청남도 천안시 동남구 버들로 34  041-521-2651  2023-08-18
3   4   충청남도 천안시 농업기술센터  충청남도 천안시 동남구 목천읍 목천안터1길 15  041-521-2962  2023-08-18
4   5  충청남도 천안시 맑은물사업본부       충청남도 천안시 동남구 용곡2길 141  041-521-3111  2023-08-18


In [ ]:
print(shelter.shape)
print(shelter.columns)
print(shelter.head())

(378, 6)
Index(['순번', '설치주소', '설치위치', '설치년도', '설치형상', '관리부서'], dtype='object')
   순번                  설치주소              설치위치    설치년도     설치형상   관리부서
0   1  충청남도 천안시 동남구 천안대로357     천안생활체육공원 산책로   2011.0      팔각정  체육진흥과
1   2  충청남도 천안시 동남구 천안대로357     천안생활체육공원 야외무대  2011.0  막구조 파고라  체육진흥과
2   3  충청남도 천안시 동남구 천안대로357  천안생활체육공원 인조잔디축구장  2012.0  막구조 파고라  체육진흥과
3   4  충청남도 천안시 동남구 천안대로357    천안생활체육공원 다목적구장  2011.0  그늘막 파고라  체육진흥과
4   5  충청남도 천안시 동남구 천안대로357    천안생활체육공원 X-게임장  2011.0  사각 파고라A  체육진흥과


In [ ]:
print(outdoor.shape)
print(outdoor.columns)
print(outdoor.head())

(272, 24)
Index(['연번', '시설명', '소재지', '설치년도', '운동기구1', '운동기구1 개수', '운동기구2', '운동기구2 개수',
       '운동기구3', '운동기구3 개수', '운동기구4', '운동기구4 개수', '운동기구5', '운동기구5 개수', '운동기구6',
       '운동기구6 개수', '운동기구7', '운동기구7 개수', '운동기구8', '운동기구8 개수', '운동기구9',
       '운동기구9 개수', '운동기구10', '운동기구10 개수'],
      dtype='object')
    연번   시설명            소재지     설치년도        운동기구1  운동기구1 개수        운동기구2  \
0  1.0  마을회관      목천읍 천정3길4  2014 이전         공중걷기         1  상체근육풀기+파도타기   
1  2.0  마을회관    목천읍 천정2길 46  2014 이전  상체근육풀기+파도타기         1        허리돌리기   
2  3.0  마을회관    목천읍 삼성3길 23     2010          달리기         1        허리돌리기   
3  4.0  마을회관     목천읍 삼성1길 7     2017    공중걷기+파도타기         1        허리돌리기   
4  5.0  마을회관  목천읍 교천3길 12-5  2014 이전     공중걷기+달리기         1        허리돌리기   

   운동기구2 개수         운동기구3  운동기구3 개수  ...   운동기구6  운동기구6 개수 운동기구7  운동기구7 개수  \
0       1.0  등허리지압기+허리지압기       1.0  ...     NaN       NaN   NaN       NaN   
1       1.0          공중걷기       1.0  ...     NaN       NaN   NaN       NaN   
2       

In [ ]:
print(facility.shape)
print(facility.columns)
print(facility.head())

(18, 4)
Index(['명칭', '주소', '주요시설', '문의처'], dtype='object')
           명칭                               주소  \
0     천안종합운동장      천안시 서북구 번영로 208(백석동, 종합운동장)   
1      한들문화센터  천안시 서북구 음봉로 861-50(백석동, 한들문화센터)   
2      천안축구센터                천안시 서북구 축구센터로 150   
3  천안시실내배드민턴장    천안시 동남구 천안대로 357(청당동 실내배드민턴장)   
4   천안생활체육야구장                 천안시 동남구 천안대로 320   

                                              주요시설           문의처  
0  주경기장+보조경기장+국민체육센터+유관순체육관+실내테니스장+실외테니스장+인라인스케이트장  041-529-5000  
1                           수영장+목욕탕+헬스장+필라테스실+이벤트홀  041-529-5177  
2                천연잔디구장+인조잔디구장+풋살구장+세미나실+운전면허학과시험장  041-529-5170  
3                                 관중석+코트+샤워실+선수대기실  041-529-5115  
4                               야구장+관리사무실+더그아웃+본부석  041-529-5119  


## 입주기업

In [ ]:
path_company = "/content/drive/MyDrive/데이터/충청남도 천안시_산업단지 입주기업 현황_20250211.CSV"

In [ ]:
company = pd.read_csv(path_company, encoding="cp949")

In [ ]:
print(company.shape)
print(company.columns)
print(company.head())

(1452, 7)
Index(['순번', '단지명', '회사명', '공장대표주소(도로명)', '공장대표주소(지번)', '업종명', '전화번호'], dtype='object')
   순번           단지명             회사명  \
0   1  천안5산단외국인투자지역  (주)아마쎌지오스에어로젤스   
1   2  천안5산단외국인투자지역       (주)엠에스씨테크   
2   3  천안5산단외국인투자지역      (주)제이씨유코리아   
3   4  천안5산단외국인투자지역      (주)코벤티아코리아   
4   5  천안5산단외국인투자지역      (주)티지케이코리아   

                                     공장대표주소(도로명)  \
0               충청남도 천안시 동남구 성남면 대화리 337번지 외 1필지   
1                     충청남도 천안시 동남구 수신면 5산단4로 110   
2                      충청남도 천안시 동남구 성남면 5산단1로 96   
3                    충청남도 천안시 동남구 성남면 5산단3로 77-8   
4  충청남도 천안시 동남구 성남면 5산단1로 136, (325,326번지) 외 1필지   

                                     공장대표주소(지번)                          업종명  \
0              충청남도 천안시 동남구 성남면 대화리 337번지 외 1필지        그 외 기타 분류 안된 화학제품 제조업   
1                    충청남도 천안시 동남구 수신면 신풍리 558번지                동주물 주조업 외 1 종   
2                    충청남도 천안시 동남구 성남면 대화리 339번지  그 외 기타 분류 안된 화학제품 제조업 외 1 종   
3                    충청남도 천안시 동남구 

In [ ]:
# 지오코딩
print("입주기업 주소 -> 좌표 변환을 시작합니다. (약 2~3분 소요)")
from tqdm.auto import tqdm
tqdm.pandas()

# geocode_kakao 함수가 이전에 정의되었다고 가정합니다.
company[['lon', 'lat']] = company['공장대표주소(도로명)'].progress_apply(
    lambda x: pd.Series(geocode_kakao(x))
)
company_geocoded = company.dropna(subset=['lat', 'lon']).copy()

print(f"✅ 좌표 변환 완료! (총 {len(company)}개 중 {len(company_geocoded)}개 성공)")

입주기업 주소 -> 좌표 변환을 시작합니다. (약 2~3분 소요)


  0%|          | 0/1452 [00:00<?, ?it/s]

✅ 좌표 변환 완료! (총 1452개 중 1305개 성공)


# 변수 설계

## 인구 데이터 전처리 및 변수 생성

In [ ]:
# ==============================================================================
# 1. 인구 데이터 로드 및 클리닝
# ==============================================================================
# 천안시 청년 인구 데이터 로드
# 첫 번째 데이터 행이 헤더이므로 skiprows=1을 사용합니다.
path_pop_cheonan = "/content/drive/MyDrive/데이터/연령_및_성별_인구_–_읍면동_20250830034743.csv"
pop_cheonan = pd.read_csv(path_pop_cheonan, encoding='cp949', skiprows=1)
pop_cheonan = pop_cheonan.iloc[1:] # 불필요한 행 제거
new_cols_cheonan = ['행정구역', '연령별'] + [str(y) for y in range(2015, 2025)]
pop_cheonan.columns = new_cols_cheonan
for col in new_cols_cheonan[2:]:
    pop_cheonan[col] = pd.to_numeric(pop_cheonan[col], errors='coerce')


# 전국 청년 인구 데이터 로드
# 데이터가 3번째 행부터 시작하므로 skiprows=2를 사용합니다.
path_pop_nationwide = "/content/drive/MyDrive/데이터/연령_및_성별_인구_–_읍면동_20250828212700_전국.csv"
pop_nationwide = pd.read_csv(path_pop_nationwide, encoding='cp949', skiprows=2)
new_cols_nationwide = ['행정구역', '연령별'] + [str(y) for y in range(2015, 2025)]
pop_nationwide.columns = new_cols_nationwide
for col in new_cols_nationwide[2:]:
    pop_nationwide[col] = pd.to_numeric(pop_nationwide[col], errors='coerce')

In [ ]:
# ==============================================================================
# 2. 청년 인구 변수 생성
# ==============================================================================
# '동남구'와 '서북구'의 청년 인구(20-29세) 합산
df_youth_gu = pop_cheonan[
    pop_cheonan['행정구역'].isin(['동남구', '서북구']) & pop_cheonan['연령별'].isin(['20~24세', '25~29세'])
].copy()
df_youth_gu = df_youth_gu.groupby('행정구역').sum(numeric_only=True).T.rename_axis('연도')
df_youth_gu.columns = [f'{col}_청년인구' for col in df_youth_gu.columns]
df_youth_gu.reset_index(inplace=True)
df_youth_gu['연도'] = df_youth_gu['연도'].astype(int)

# '천안시' 청년 인구 계산 (동남구 + 서북구)
df_youth_gu['천안시_청년인구'] = df_youth_gu['동남구_청년인구'] + df_youth_gu['서북구_청년인구']


# 전국 청년 인구 합산
df_youth_nationwide = pop_nationwide[
    pop_nationwide['연령별'].isin(['20~24세', '25~29세'])
].copy()
df_youth_nationwide = df_youth_nationwide.sum(numeric_only=True).to_frame(name='전국_청년인구')
df_youth_nationwide.reset_index(inplace=True)
df_youth_nationwide.columns = ['연도', '전국_청년인구']
df_youth_nationwide['연도'] = df_youth_nationwide['연도'].astype(int)

In [ ]:
# ==============================================================================
# 3. 전체 인구 변수 생성 및 비율 계산
# ==============================================================================
# 천안시 전체 인구 추출: 모든 연령대 데이터를 합산
df_total_gu = pop_cheonan[
    pop_cheonan['행정구역'].isin(['동남구', '서북구'])
].copy()
df_total_gu = df_total_gu.groupby('행정구역').sum(numeric_only=True).T.rename_axis('연도')
df_total_gu.columns = [f'{col}_전체인구' for col in df_total_gu.columns]
df_total_gu.reset_index(inplace=True)
df_total_gu['연도'] = df_total_gu['연도'].astype(int)

# '천안시' 전체 인구 계산 (동남구 + 서북구)
df_total_gu['천안시_전체인구'] = df_total_gu['동남구_전체인구'] + df_total_gu['서북구_전체인구']

# 전국 전체 인구 추출: 모든 연령대 데이터를 합산
df_total_nationwide = pop_nationwide.sum(numeric_only=True).to_frame(name='전국_전체인구')
df_total_nationwide.reset_index(inplace=True)
df_total_nationwide.columns = ['연도', '전국_전체인구']
df_total_nationwide['연도'] = df_total_nationwide['연도'].astype(int)

In [ ]:
# ==============================================================================
# 4. 최종 데이터프레임 병합 및 청년 인구 비율 계산
# ==============================================================================
# 청년 인구 데이터와 전체 인구 데이터를 병합
df_final = pd.merge(df_youth_gu, df_total_gu, on='연도')

# 전국 청년 인구 데이터 병합
df_final = pd.merge(df_final, df_youth_nationwide, on='연도')

# 전국 전체 인구 데이터 병합
df_final = pd.merge(df_final, df_total_nationwide, on='연도')

# 인구 비율 계산
df_final['천안시_청년비율'] = df_final['천안시_청년인구'] / df_final['전국_전체인구']
df_final['동남구_청년비율'] = df_final['동남구_청년인구'] / df_final['전국_전체인구']
df_final['서북구_청년비율'] = df_final['서북구_청년인구'] / df_final['전국_전체인구']

# 칼럼 순서 재정의
df_final = df_final[[
    '연도', '천안시_청년인구', '천안시_청년비율',
    '동남구_청년인구', '동남구_청년비율',
    '서북구_청년인구', '서북구_청년비율',
    '전국_청년인구', '전국_전체인구'
]]

print("--- 천안시 및 전국 청년 인구 지표 ---")
print(df_final.head())

--- 천안시 및 전국 청년 인구 지표 ---
     연도  천안시_청년인구  천안시_청년비율  동남구_청년인구  동남구_청년비율  서북구_청년인구  서북구_청년비율  전국_청년인구  \
0  2015     92587  0.030578     40650  0.013425     51937  0.017153  3027896   
1  2016     93703  0.030532     40439  0.013177     53264  0.017356  3068970   
2  2017     97230  0.030730     40731  0.012873     56499  0.017857  3164039   
3  2018     98705  0.030284     41215  0.012645     57490  0.017639  3259284   
4  2019     99695  0.029818     42010  0.012565     57685  0.017253  3343423   

   전국_전체인구  
0  3027896  
1  3068970  
2  3164039  
3  3259284  
4  3343423  


## 상권 데이터 전처리 및 변수 생성

In [ ]:
# 1. 천안시 동남구/서북구만 필터링
consume_cheonan = consume[consume['시군구코드'].isin([44131, 44133])].copy()
# 시군구코드 -> 행정동명 매핑
code_to_dong = {44131: '동남구', 44133: '서북구'}
consume_cheonan['행정구명'] = consume_cheonan['시군구코드'].map(code_to_dong)

In [ ]:
# 2. 업종 다양성 지수 (HHI 역수) 함수
def calculate_hhi_inverse(df, col):
    if df.empty:
        return 0
    counts = df[col].value_counts(normalize=True)
    hhi = (counts ** 2).sum()
    return 1 / hhi if hhi > 0 else 0

In [ ]:
# 3. 행정구별 업종 다양성 지수 계산
diversity_gu = consume_cheonan.groupby('행정구명').apply(
    calculate_hhi_inverse, '상권업종중분류명'
).reset_index(name='업종다양성지수')

youth_density_gu = consume_cheonan[consume_cheonan['상권업종대분류명'].isin(['음식', '숙박', '음료'])]
youth_density_gu = youth_density_gu.groupby('행정구명').size().reset_index(name='청년친화업종수')

biz_df_gu = pd.merge(diversity_gu, youth_density_gu, on='행정구명', how='left')
biz_df_gu.fillna(0, inplace=True) # 결측값 0으로 채우기

In [ ]:
# 4. 읍면동별 업종 지표 계산
# 원본 데이터에 '행정동명' 칼럼이 있다고 가정합니다. 없다면 다른 칼럼명을 사용해야 합니다.
consume_cheonan.rename(columns={'읍면동명': '행정동명'}, inplace=True)

diversity_dong = consume_cheonan.groupby('행정동명').apply(
    calculate_hhi_inverse, '상권업종중분류명'
).reset_index(name='업종다양성지수')

youth_density_dong = consume_cheonan[consume_cheonan['상권업종대분류명'].isin(['음식', '숙박', '음료'])]
youth_density_dong = youth_density_dong.groupby('행정동명').size().reset_index(name='청년친화업종수')

biz_df_dong = pd.merge(diversity_dong, youth_density_dong, on='행정동명', how='left')
biz_df_dong.fillna(0, inplace=True)

## 교통/시설 데이터 전처리 및 변수 생성

In [ ]:
# 카카오 API(개인 API 발급 후 넣어야 됨.)
API_KEY = "372f670d9a0080e1106d8ac37afde6dc"
def geocode_kakao(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {API_KEY}"}
    try:
        response = requests.get(url, headers=headers, params={"query": address})
        response.raise_for_status()
        doc = response.json().get('documents')
        if doc and len(doc) > 0:
            return float(doc[0].get('x')), float(doc[0].get('y'))
    except Exception as e:
        print(f"Error geocoding '{address}': {e}")
    return None, None

In [ ]:
# 1. 주소 칼럼을 '주소'로 통일
if '소재지도로명주소' in admin.columns:
    admin = admin.rename(columns={'소재지도로명주소': '주소'})
else:
    admin = pd.DataFrame(columns=['주소'])

if '설치주소' in outdoor.columns:
    outdoor = outdoor.rename(columns={'설치주소': '주소'})
else:
    outdoor = pd.DataFrame(columns=['주소'])

if '소재지' in shelter.columns:
    shelter = shelter.rename(columns={'소재지': '주소'})
else:
    shelter = pd.DataFrame(columns=['주소'])

if '주소' not in facility.columns:
    facility = pd.DataFrame(columns=['주소'])

In [ ]:
# 2. 모든 시설물 데이터를 하나의 데이터프레임으로 결합
all_facilities = pd.concat([admin[['주소']], outdoor[['주소']], shelter[['주소']], facility[['주소']]])
all_facilities.dropna(subset=['주소'], inplace=True)
all_facilities.drop_duplicates(inplace=True)

In [ ]:
# 3. 주소에서 행정구역 및 읍면동명 추출
all_facilities['행정구명'] = all_facilities['주소'].str.extract(r'(동남구|서북구)')
# 수정: '숫자'와 '괄호'를 제외하고 '읍', '면', '동'으로 끝나는 단어만 추출
all_facilities['행정동명'] = all_facilities['주소'].str.extract(r'([가-힣]+읍|[가-힣]+면|[가-힣]+동)')

# 추출된 값이 없는 경우 제거
facilities_with_district = all_facilities.dropna(subset=['행정구명']).copy()
facilities_with_dong = all_facilities.dropna(subset=['행정동명']).copy()

In [ ]:
# 4. 시설물 수 집계
facilities_by_gu = facilities_with_district.groupby('행정구명').size().reset_index(name='여가 혹은 행정시설물수')
facilities_by_dong = facilities_with_dong.groupby('행정동명').size().reset_index(name='여가 혹은 행정시설물수')

print("--- 주소 기반 구별 시설물 집계 ---")
print(facilities_by_gu)
print("\n--- 주소 기반 읍면동별 시설물 집계 ---")
print(facilities_by_dong.head())

--- 주소 기반 구별 시설물 집계 ---
  행정구명  여가 혹은 행정시설물수
0  동남구            32
1  서북구            25

--- 주소 기반 읍면동별 시설물 집계 ---
  행정동명  여가 혹은 행정시설물수
0  광덕면             2
1  대흥동             1
2   동면             1
3  목천읍             3
4  백석동             3


## 캠퍼스/기숙사와 교통 강화 파생변수

In [ ]:
# 경계 파일
path_dong = "/content/drive/MyDrive/데이터/LSMD_ADM_SECT_UMD_충남/LSMD_ADM_SECT_UMD_44_202506.shp"

# 1) 읽기 + 좌표계
dong_map = gpd.read_file(path_dong, encoding="cp949").to_crs(4326)

In [ ]:
# 2) 표준 컬럼 설정
EMD_NAME_COL = "EMD_NM"   # 읍면동 이름
# 핵심: 구 코드(동남/서북)는 EMD_CD 앞 5자리에서 만든다
dong_map["SIG_FROM_EMD"] = dong_map["EMD_CD"].astype(str).str.slice(0, 5)
SIG_CD_COL = "SIG_FROM_EMD"

# 확인(참고)
print(dong_map[[EMD_NAME_COL, "EMD_CD", "COL_ADM_SE", SIG_CD_COL]].head())
print("COL_ADM_SE 분포:", dong_map["COL_ADM_SE"].astype(str).value_counts().head())
print("SIG_FROM_EMD 분포:", dong_map[SIG_CD_COL].value_counts().head())

  EMD_NM    EMD_CD COL_ADM_SE SIG_FROM_EMD
0    원성동  44131107      44130        44131
1    풍세면  44131310      44130        44131
2    문화동  44131103      44130        44131
3    용곡동  44131115      44130        44131
4    불당동  44133108      44130        44133
COL_ADM_SE 분포: COL_ADM_SE
44130    43
44150    37
44200    30
44230    24
44210    24
Name: count, dtype: int64
SIG_FROM_EMD 분포: SIG_FROM_EMD
44150    37
44200    30
44131    28
44210    24
44230    24
Name: count, dtype: int64


In [ ]:
# 3) 천안(동남/서북)만 필터
dong_map_cheon = dong_map[dong_map[SIG_CD_COL].isin(["44131","44133"])].copy()
print("[INFO] 천안 읍면동 수:", len(dong_map_cheon))
print(dong_map_cheon[[EMD_NAME_COL, SIG_CD_COL]].head())

[INFO] 천안 읍면동 수: 43
  EMD_NM SIG_FROM_EMD
0    원성동        44131
1    풍세면        44131
2    문화동        44131
3    용곡동        44131
4    불당동        44133


In [ ]:
# -----------------------------
# 0) 캠퍼스 좌표 생성 (geocode_kakao가 (x, y) = (경도, 위도) 튜플 반환한다고 가정)
# -----------------------------
univ_addresses = [
    ("단국대학교", "충남 천안시 동남구 단대로 119"),
    ("상명대학교", "충남 천안시 동남구 상명대길 31"),
    ("백석대학교", "충남 천안시 동남구 백석대학로 1-1"),
    ("나사렛대학교", "충남 천안시 서북구 월봉로 48"),
    ("남서울대학교", "충남 천안시 서북구 성환읍 대학로 91"),
    ("호서대학교", "충남 천안시 동남구 호서대길 12"),
    ("한국기술교육대학교", "충남 천안시 동남구 병천면 충절로 1600"),
    ("한국기술교육대학교", "충남 천안시 서북구 과수원길 18"),
]

rows = []
for std_name, addr in univ_addresses:
    x, y = geocode_kakao(addr)        # (lon, lat)
    rows.append({"학교_std": std_name, "addr": addr, "lat": y, "lon": x})

df_geo_ok = pd.DataFrame(rows).dropna(subset=["lat","lon"]).copy()
df_geo_ok["lat"] = pd.to_numeric(df_geo_ok["lat"], errors="coerce")
df_geo_ok["lon"] = pd.to_numeric(df_geo_ok["lon"], errors="coerce")
df_geo_ok = df_geo_ok.dropna(subset=["lat","lon"]).copy()

In [ ]:
# -----------------------------
# 1) 캠퍼스 → 행정동 매핑 (학교_std로 유지)
# -----------------------------
campus_gdf = gpd.GeoDataFrame(
    df_geo_ok,
    geometry=gpd.points_from_xy(df_geo_ok["lon"], df_geo_ok["lat"]),
    crs="EPSG:4326"
)

campus_join = gpd.sjoin(
    campus_gdf[["학교_std","addr","geometry"]],
    dong_map_cheon[[EMD_NAME_COL,"geometry"]].rename(columns={EMD_NAME_COL:"행정동명"}),
    how="left", predicate="within"
)[["학교_std","addr","행정동명"]]

print("[캠퍼스-동 매핑]"); print(campus_join)

[캠퍼스-동 매핑]
      학교_std                     addr 행정동명
0      단국대학교       충남 천안시 동남구 단대로 119  신부동
1      상명대학교       충남 천안시 동남구 상명대길 31  안서동
2      백석대학교     충남 천안시 동남구 백석대학로 1-1  안서동
3     나사렛대학교        충남 천안시 서북구 월봉로 48  쌍용동
4     남서울대학교    충남 천안시 서북구 성환읍 대학로 91  성환읍
5      호서대학교       충남 천안시 동남구 호서대길 12  안서동
6  한국기술교육대학교  충남 천안시 동남구 병천면 충절로 1600  병천면
7  한국기술교육대학교       충남 천안시 서북구 과수원길 18  부대동


In [ ]:
# -----------------------------
# 2) 기숙사: 학교명 표준화 후 동으로 집계
# -----------------------------
# (1) 학교명 정규화(공백/캠퍼스 꼬리표 제거 → 학교_std)
import re

dorm_x = dorm_clean.copy()
dorm_x["학교_raw"]  = dorm_x["학교"].astype(str)
dorm_x["학교_norm"] = dorm_x["학교_raw"].str.strip().str.replace(r"\s+", "", regex=True)

def to_std(s):
    if s is None: return None
    # 캠퍼스 꼬리표가 있어도 표준 대학명으로 매핑
    patterns = [
        (r'단국대학교.*', '단국대학교'),
        (r'상명대학교.*', '상명대학교'),
        (r'남서울대학교.*', '남서울대학교'),
        (r'백석대학교.*', '백석대학교'),
        (r'호서대학교.*', '호서대학교'),
        (r'한국기술교육대학교.*', '한국기술교육대학교'),
        (r'나사렛대학교.*', '나사렛대학교'),
    ]
    for pat, std in patterns:
        if re.fullmatch(pat, s):
            return std
    return s

dorm_x["학교_std"] = dorm_x["학교_norm"].apply(to_std)

# (2) 천안 대상만 필터
cheonan_std = {'단국대학교','상명대학교','남서울대학교','백석대학교','호서대학교','한국기술교육대학교','나사렛대학교'}
dorm_cheonan_std = dorm_x[dorm_x["학교_std"].isin(cheonan_std)].copy()

# (3) 학교×연도 수용인원 합
dorm_by_school = dorm_cheonan_std.groupby(["기준연도","학교_std"], as_index=False)["실제 수용 인원"].sum()

# (4) 캠퍼스-동 매핑 붙이기(학교_std 기준!)
dorm_with_dong = pd.merge(dorm_by_school, campus_join[["학교_std","행정동명"]], on="학교_std", how="left")

# (5) 최신연도 대표값 → 동별 합
if dorm_with_dong["기준연도"].notna().any():
    latest_year = dorm_with_dong["기준연도"].max()
    dorm_latest = (dorm_with_dong[dorm_with_dong["기준연도"]==latest_year]
                   .groupby("행정동명", as_index=False)["실제 수용 인원"].sum()
                   .rename(columns={"실제 수용 인원":"기숙사수용인원"}))
else:
    dorm_latest = dorm_with_dong.groupby("행정동명", as_index=False)["실제 수용 인원"].sum() \
                                 .rename(columns={"실제 수용 인원":"기숙사수용인원"})

dorm_latest["기숙사수용인원"] = dorm_latest["기숙사수용인원"].fillna(0)

print("[기숙사-동 최신]"); print(dorm_latest.head())

[기숙사-동 최신]
  행정동명  기숙사수용인원
0  병천면   2400.0
1  부대동   2400.0
2  성환읍   1126.0
3  신부동   1966.0
4  쌍용동    984.0


In [ ]:
# -----------------------------
# 3) 상권: 동으로 점포수/다양성 집계
# -----------------------------
shops = consume_cheonan.copy()
for c in ["위도","경도"]:
    shops[c] = pd.to_numeric(shops[c], errors="coerce")
shops = shops.dropna(subset=["위도","경도"]).copy()

shops_gdf = gpd.GeoDataFrame(
    shops,
    geometry=gpd.points_from_xy(shops["경도"], shops["위도"]),
    crs="EPSG:4326"
)

shops_in_dong = gpd.sjoin(
    shops_gdf[["상호명","상권업종대분류명","상권업종중분류명","상권업종소분류명","geometry"]],
    dong_map_cheon[[EMD_NAME_COL,"geometry"]].rename(columns={EMD_NAME_COL:"행정동명"}),
    how="left", predicate="within"
)

shop_cnt = shops_in_dong.groupby("행정동명").size().reset_index(name="점포수")
div_col = "상권업종소분류명" if "상권업종소분류명" in shops_in_dong.columns else (
          "상권업종중분류명" if "상권업종중분류명" in shops_in_dong.columns else "상권업종대분류명")
shop_div = shops_in_dong.groupby("행정동명")[div_col].nunique().reset_index(name="업종다양성")

In [ ]:
# -----------------------------
# 4) 교통: 좌표 없으면 자동 생략(=NaN → 0으로 보정)
# -----------------------------
from sklearn.preprocessing import MinMaxScaler
import numpy as np

if {"lat","lon"}.issubset(stops.columns):
    stops2 = stops.dropna(subset=["lat","lon"]).copy()
    stops2["lat"] = pd.to_numeric(stops2["lat"], errors="coerce")
    stops2["lon"] = pd.to_numeric(stops2["lon"], errors="coerce")
    stops2 = stops2.dropna(subset=["lat","lon"])

    if not stops2.empty:
        stops_gdf = gpd.GeoDataFrame(
            stops2, geometry=gpd.points_from_xy(stops2["lon"], stops2["lat"]), crs="EPSG:4326"
        )
        # 면적(km2)
        dong_area = dong_map_cheon[[EMD_NAME_COL,"geometry"]].rename(columns={EMD_NAME_COL:"행정동명"})
        dong_area_m = dong_area.to_crs(5179)
        dong_area["면적_km2"] = dong_area_m.area / 1e6

        # 정류장 수/밀도
        stops_in_dong = gpd.sjoin(
            stops_gdf[["geometry"]],
            dong_map_cheon[[EMD_NAME_COL,"geometry"]].rename(columns={EMD_NAME_COL:"행정동명"}),
            how="left", predicate="within"
        )
        stop_cnt = stops_in_dong.groupby("행정동명").size().reset_index(name="정류장수")
        stop_density = dong_area.merge(stop_cnt, on="행정동명", how="left")
        stop_density["정류장수"] = stop_density["정류장수"].fillna(0)
        stop_density["정류장밀도"] = stop_density["정류장수"] / stop_density["면적_km2"].replace(0, np.nan)

        # 근접성(동 중심 → 최근접 정류장, 위경도 차 근사)
        dong_cent = dong_map_cheon[[EMD_NAME_COL,"geometry"]].rename(columns={EMD_NAME_COL:"행정동명"}).copy()
        dong_cent["centroid"] = dong_cent.to_crs(5179).centroid.to_crs(4326)
        stops_xy = stops_gdf[["lon","lat"]].to_numpy()

        def _nearest_deg(row):
            if len(stops_xy)==0: return np.nan
            lon0, lat0 = row["centroid"].x, row["centroid"].y
            d = np.sqrt((stops_xy[:,0]-lon0)**2 + (stops_xy[:,1]-lat0)**2)
            return d.min()

        dong_cent["최근접정류장_근사거리"] = dong_cent.apply(_nearest_deg, axis=1)

        # 교통접근성지수
        tdf = stop_density.merge(dong_cent[["행정동명","최근접정류장_근사거리"]], on="행정동명", how="left")
        sc1, sc2 = MinMaxScaler(), MinMaxScaler()
        tdf["정류장밀도_norm"] = sc1.fit_transform(tdf[["정류장밀도"]].fillna(0))
        tdf["근접성_norm"]     = 1 - sc2.fit_transform(tdf[["최근접정류장_근사거리"]])
        tdf["교통접근성지수"]   = 0.6*tdf["정류장밀도_norm"] + 0.4*tdf["근접성_norm"]
        transport_idx = tdf[["행정동명","교통접근성지수"]]
    else:
        transport_idx = pd.DataFrame({"행정동명": dong_map_cheon[EMD_NAME_COL].unique(), "교통접근성지수": np.nan})
else:
    transport_idx = pd.DataFrame({"행정동명": dong_map_cheon[EMD_NAME_COL].unique(), "교통접근성지수": np.nan})

## 최종 데이터프레임

In [ ]:
# -----------------------------
# 0) 동 → 구 매핑 테이블 만들기
# -----------------------------
# dong_map_cheon: [EMD_NM(=EMD_NAME_COL), SIG_FROM_EMD] 보유
dong_to_gu = dong_map_cheon[[EMD_NAME_COL, SIG_CD_COL]].copy()
dong_to_gu["행정동명"] = dong_to_gu[EMD_NAME_COL].astype(str)
dong_to_gu["구코드"] = dong_to_gu[SIG_CD_COL].astype(str)
code2gu = {"44131": "동남구", "44133": "서북구"}
dong_to_gu["행정구명"] = dong_to_gu["구코드"].map(code2gu)
dong_to_gu = dong_to_gu[["행정동명","행정구명"]].dropna()

# -------------------------------------------------
# 1) 구(區) 단위 최종 DF: 기존 틀 + (동→구) 교통/기숙사 집계 추가
# -------------------------------------------------
# (1) 기존 인구 프레임을 long으로 (질문 코드 그대로)
df_pop_long_gu = pd.DataFrame()
for gu in ["동남구","서북구"]:
    tmp = df_final[["연도", f"{gu}_청년인구", f"{gu}_청년비율"]].copy()
    tmp = tmp.rename(columns={f"{gu}_청년인구":"청년인구", f"{gu}_청년비율":"청년비율"})
    tmp["행정구명"] = gu
    df_pop_long_gu = pd.concat([df_pop_long_gu, tmp], ignore_index=True)

# (2) 상권·시설(구) 붙이기 (질문 코드 그대로)
df_final_long_gu = (df_pop_long_gu
                    .merge(biz_df_gu,          on="행정구명", how="left")
                    .merge(facilities_by_gu,   on="행정구명", how="left"))

# (3) 시 단위(천안시)도 함께 붙이기 (질문 코드 그대로)
df_final_gu_merged = df_final_long_gu.merge(
    df_final[["연도","천안시_청년인구","천안시_청년비율"]],
    on="연도", how="left"
)

# (4) 동별 교통/기숙사 → 구별 집계
#     - 교통접근성지수: 동 지수의 "평균" (대안: 면적/인구 가중 평균)
#     - 기숙사수용인원: 동 합계
# 준비: 동별 테이블 두 개를 구 명칭 부여
transport_by_dong = transport_idx.copy()
if "행정동명" not in transport_by_dong.columns:
    transport_by_dong = transport_by_dong.rename(columns={"동":"행정동명"})
transport_by_dong = transport_by_dong.merge(dong_to_gu, on="행정동명", how="left")

dorm_by_dong = dorm_latest.copy()
if "행정동명" not in dorm_by_dong.columns:
    dorm_by_dong = dorm_by_dong.rename(columns={"동":"행정동명"})
dorm_by_dong = dorm_by_dong.merge(dong_to_gu, on="행정동명", how="left")

# 집계
gu_transport = (transport_by_dong
                .groupby("행정구명", as_index=False)["교통접근성지수"]
                .mean())  # 평균

gu_dorm = (dorm_by_dong
           .groupby("행정구명", as_index=False)["기숙사수용인원"]
           .sum())     # 합계

# (5) 구 단위에 교통/기숙사 붙이기
df_final_gu_merged = (df_final_gu_merged
                      .merge(gu_transport, on="행정구명", how="left")
                      .merge(gu_dorm,      on="행정구명", how="left"))

# 결측 0 보정
for c in ["업종다양성지수","청년친화업종수","여가 혹은 행정시설물수","교통접근성지수","기숙사수용인원"]:
    if c in df_final_gu_merged.columns:
        df_final_gu_merged[c] = pd.to_numeric(df_final_gu_merged[c], errors="coerce").fillna(0)

# (6) 구 단위 지수 산출 (정규화 + 가중합)
cols_scale_gu = ["업종다양성지수","청년친화업종수","여가 혹은 행정시설물수","교통접근성지수","기숙사수용인원"]
exist_cols_gu = [c for c in cols_scale_gu if c in df_final_gu_merged.columns]
sc_gu = MinMaxScaler()
scaled = sc_gu.fit_transform(df_final_gu_merged[exist_cols_gu])
for i, c in enumerate(exist_cols_gu):
    df_final_gu_merged[c + "_norm"] = scaled[:, i]

# 가중치 예시(구 단위): 교통 0.4, 기숙사 0.3, 업다 0.2, 청친 0.05, 시설 0.05
def _safe(col): return df_final_gu_merged.get(col, pd.Series(0, index=df_final_gu_merged.index))
df_final_gu_merged["모빌리티지수_구"] = (
    0.4*_safe("교통접근성지수_norm") +
    0.3*_safe("기숙사수용인원_norm") +
    0.2*_safe("업종다양성지수_norm") +
    0.05*_safe("청년친화업종수_norm") +
    0.05*_safe("여가 혹은 행정시설물수_norm")
).round(3)

# 등급(3분위)
q1, q2 = df_final_gu_merged["모빌리티지수_구"].quantile([0.33, 0.66])
df_final_gu_merged["등급_구"] = pd.cut(
    df_final_gu_merged["모빌리티지수_구"],
    bins=[-1, q1, q2, 1.01],
    labels=["주의","보통","우수"]
)

print("\n--- 구(區) 단위 최종 데이터프레임 ---")
print(df_final_gu_merged.head())


--- 구(區) 단위 최종 데이터프레임 ---
     연도   청년인구      청년비율 행정구명    업종다양성지수  청년친화업종수  여가 혹은 행정시설물수  천안시_청년인구  \
0  2015  40650  0.013425  동남구  23.778435     4750            32     92587   
1  2016  40439  0.013177  동남구  23.778435     4750            32     93703   
2  2017  40731  0.012873  동남구  23.778435     4750            32     97230   
3  2018  41215  0.012645  동남구  23.778435     4750            32     98705   
4  2019  42010  0.012565  동남구  23.778435     4750            32     99695   

   천안시_청년비율  교통접근성지수  기숙사수용인원  업종다양성지수_norm  청년친화업종수_norm  여가 혹은 행정시설물수_norm  \
0  0.030578      0.0   7657.0           1.0           0.0                1.0   
1  0.030532      0.0   7657.0           1.0           0.0                1.0   
2  0.030730      0.0   7657.0           1.0           0.0                1.0   
3  0.030284      0.0   7657.0           1.0           0.0                1.0   
4  0.029818      0.0   7657.0           1.0           0.0                1.0   

   교통접근성지수_norm  기숙사수용인원_norm

In [ ]:
# -------------------------------------------------
# 2) 읍면동(洞) 단위 최종 DF: 기존 틀 + 교통/기숙사 붙이고 지수
# -------------------------------------------------
# (1) 기존 질문 코드(상권·시설 결합)
df_final_eupmyeondong = pd.merge(biz_df_dong, facilities_by_dong, on='행정동명', how='outer')
df_final_eupmyeondong.fillna(0, inplace=True)

# (2) 교통/기숙사(동별) 붙이기
df_final_eupmyeondong = (df_final_eupmyeondong
                         .merge(transport_idx, on="행정동명", how="left")
                         .merge(dorm_latest,   on="행정동명", how="left"))

for c in ["업종다양성지수","청년친화업종수","여가 혹은 행정시설물수","교통접근성지수","기숙사수용인원"]:
    if c in df_final_eupmyeondong.columns:
        df_final_eupmyeondong[c] = pd.to_numeric(df_final_eupmyeondong[c], errors="coerce").fillna(0)

# (3) 동 단위 지수 산출 (정규화 + 가중합) — 동에서는 업다/시설 가중 조금 더 주는 버전
cols_scale_dong = ["업종다양성지수","청년친화업종수","여가 혹은 행정시설물수","교통접근성지수","기숙사수용인원"]
exist_cols_dong = [c for c in cols_scale_dong if c in df_final_eupmyeondong.columns]
sc_d = MinMaxScaler()
scaled_d = sc_d.fit_transform(df_final_eupmyeondong[exist_cols_dong])
for i, c in enumerate(exist_cols_dong):
    df_final_eupmyeondong[c + "_norm"] = scaled_d[:, i]

# 가중치 예시(동 단위): 교통 0.4, 기숙사 0.3, 업다 0.2, 청친 0.05, 시설 0.05
def _safe_d(col): return df_final_eupmyeondong.get(col, pd.Series(0, index=df_final_eupmyeondong.index))
df_final_eupmyeondong["청년캠퍼스모빌리티지수"] = (
    0.4*_safe_d("교통접근성지수_norm") +
    0.3*_safe_d("기숙사수용인원_norm") +
    0.2*_safe_d("업종다양성지수_norm") +
    0.05*_safe_d("청년친화업종수_norm") +
    0.05*_safe_d("여가 혹은 행정시설물수_norm")
).round(3)

q1d, q2d = df_final_eupmyeondong["청년캠퍼스모빌리티지수"].quantile([0.33, 0.66])
df_final_eupmyeondong["등급"] = pd.cut(
    df_final_eupmyeondong["청년캠퍼스모빌리티지수"],
    bins=[-1, q1d, q2d, 1.01],
    labels=["주의","보통","우수"]
)

print("\n--- 읍면동(洞) 단위 최종 데이터프레임 ---")
print(df_final_eupmyeondong.head())
print(df_final_eupmyeondong.columns)


--- 읍면동(洞) 단위 최종 데이터프레임 ---
  행정동명    업종다양성지수  청년친화업종수  여가 혹은 행정시설물수  교통접근성지수  기숙사수용인원  업종다양성지수_norm  \
0  광덕면  10.756801    131.0           2.0      0.0      0.0      0.401952   
1  대흥동   0.000000      0.0           1.0      0.0      0.0      0.000000   
2   동면  13.846154     24.0           1.0      0.0      0.0      0.517393   
3  목천읍  16.655737    399.0           3.0      0.0      0.0      0.622379   
4  문성동  26.761407    112.0           0.0      0.0      0.0      1.000000   

   청년친화업종수_norm  여가 혹은 행정시설물수_norm  교통접근성지수_norm  기숙사수용인원_norm  청년캠퍼스모빌리티지수  \
0      0.122316           0.666667           0.0           0.0        0.120   
1      0.000000           0.333333           0.0           0.0        0.017   
2      0.022409           0.333333           0.0           0.0        0.121   
3      0.372549           1.000000           0.0           0.0        0.193   
4      0.104575           0.000000           0.0           0.0        0.205   

   등급  
0  주의  
1  주의  
2  주의  
3  우수  

# 모델링 및 지수 산출

In [ ]:
# ==============================================================================
# 1. 지수 산출을 위한 변수 준비 및 정규화
# ==============================================================================
# 각 구의 유일한 값을 추출하여 별도 데이터프레임으로 만듭니다.
unique_gu_data = df_final_gu_merged[['행정구명', '업종다양성지수', '청년친화업종수', '여가 혹은 행정시설물수']].drop_duplicates()
unique_gu_data = unique_gu_data.reset_index(drop=True)

# Min-Max Scaler를 사용하여 변수들을 0~1 사이로 정규화합니다.
scaler = MinMaxScaler()
unique_gu_data[['업종다양성지수', '청년친화업종수', '여가 혹은 행정시설물수']] = scaler.fit_transform(unique_gu_data[['업종다양성지수', '청년친화업종수', '여가 혹은 행정시설물수']])

print("--- 정규화된 구별 데이터 ---")
print(unique_gu_data.head())

--- 정규화된 구별 데이터 ---
  행정구명  업종다양성지수  청년친화업종수  여가 혹은 행정시설물수
0  동남구      1.0      0.0           1.0
1  서북구      0.0      1.0           0.0


In [ ]:
# ==============================================================================
# 2. 청년생활권지수 산출 및 최종 데이터프레임 병합
# ==============================================================================
weights = {
    '업종다양성지수': 0.3,
    '청년친화업종수': 0.4,
    '여가 혹은 행정시설물수': 0.3
}

# 정규화된 데이터에 가중치를 곱하여 '청년생활권지수'를 산출합니다.
unique_gu_data['청년생활권지수'] = (
    unique_gu_data['업종다양성지수'] * weights['업종다양성지수'] +
    unique_gu_data['청년친화업종수'] * weights['청년친화업종수'] +
    unique_gu_data['여가 혹은 행정시설물수'] * weights['여가 혹은 행정시설물수']
)

# 최종 데이터프레임에 지수 값을 병합합니다.
df_final_result = pd.merge(
    df_final_gu_merged,
    unique_gu_data[['행정구명', '청년생활권지수']],
    on='행정구명',
    how='left'
)

# 최종 결과 확인
print("\n--- 최종 산출된 청년생활권지수 포함 데이터프레임 ---")
print(df_final_result.head())
print(df_final_result.columns)


--- 최종 산출된 청년생활권지수 포함 데이터프레임 ---
     연도   청년인구      청년비율 행정구명    업종다양성지수  청년친화업종수  여가 혹은 행정시설물수  천안시_청년인구  \
0  2015  40650  0.013425  동남구  23.778435     4750            32     92587   
1  2016  40439  0.013177  동남구  23.778435     4750            32     93703   
2  2017  40731  0.012873  동남구  23.778435     4750            32     97230   
3  2018  41215  0.012645  동남구  23.778435     4750            32     98705   
4  2019  42010  0.012565  동남구  23.778435     4750            32     99695   

   천안시_청년비율  교통접근성지수  기숙사수용인원  업종다양성지수_norm  청년친화업종수_norm  여가 혹은 행정시설물수_norm  \
0  0.030578      0.0   7657.0           1.0           0.0                1.0   
1  0.030532      0.0   7657.0           1.0           0.0                1.0   
2  0.030730      0.0   7657.0           1.0           0.0                1.0   
3  0.030284      0.0   7657.0           1.0           0.0                1.0   
4  0.029818      0.0   7657.0           1.0           0.0                1.0   

   교통접근성지수_norm  기숙사수용

# 예측지도

In [ ]:
# =========================
# 0) 준비: 입력 DF 검사/정리
# =========================
# df_final_eupmyeondong: ['행정동명','청년캠퍼스모빌리티지수','등급', (옵션:'업종다양성지수','청년친화업종수','시설물수','교통접근성지수','기숙사수용인원')]
need_cols = {"행정동명","청년캠퍼스모빌리티지수"}
missing = need_cols - set(df_final_eupmyeondong.columns)
if missing:
    raise ValueError(f"df_final_eupmyeondong에 필수 컬럼 없음: {missing}")

df_scores = df_final_eupmyeondong.copy()
df_scores["행정동명"] = df_scores["행정동명"].astype(str).str.strip()
if "등급" not in df_scores.columns:
    # 등급 없으면 3분위로 생성
    q1, q2 = df_scores["청년캠퍼스모빌리티지수"].quantile([0.33, 0.66])
    df_scores["등급"] = pd.cut(
        df_scores["청년캠퍼스모빌리티지수"], bins=[-1, q1, q2, np.inf],
        labels=["주의","보통","우수"]
    )

# dong_map_cheon: 천안 읍/면/동 경계 (EPSG:4326), EMD_NAME_COL: 동명이 들어있는 컬럼명
assert EMD_NAME_COL in dong_map_cheon.columns, f"{EMD_NAME_COL} 컬럼이 dong_map_cheon에 필요합니다."
dong_gdf = dong_map_cheon[[EMD_NAME_COL, "geometry"]].copy()
dong_gdf[EMD_NAME_COL] = dong_gdf[EMD_NAME_COL].astype(str).str.strip()

In [ ]:
# =========================
# 1) 동 중심점(센트로이드) 구하기
# =========================
# 투영계로 한번 바꿔서 센트로이드 구한 뒤 WGS84로 재투영(왜곡↓)
dong_cent = dong_gdf.to_crs(5179).copy()
dong_cent["centroid"] = dong_cent.geometry.centroid
dong_cent = dong_cent.set_geometry("centroid").to_crs(4326)
dong_cent["lat"] = dong_cent.geometry.y
dong_cent["lon"] = dong_cent.geometry.x
dong_cent = dong_cent[[EMD_NAME_COL, "lat", "lon"]].rename(columns={EMD_NAME_COL:"행정동명"})

In [ ]:
# =========================
# 2) 점표시용 DF 결합
# =========================
pts = pd.merge(df_scores, dong_cent, on="행정동명", how="left")
pts = pts.dropna(subset=["lat","lon"]).copy()

# 반경 스케일(동적): 7~20px
v = pts["청년캠퍼스모빌리티지수"].astype(float)
vmin = np.nanquantile(v, 0.05) if np.isfinite(v).all() else v.min()
vmax = np.nanquantile(v, 0.95) if np.isfinite(v).all() else v.max()
if not np.isfinite(vmin) or not np.isfinite(vmax) or vmin==vmax:
    vmin, vmax = 0.0, 1.0
size = 7 + ( (v.clip(vmin, vmax) - vmin) / (vmax - vmin + 1e-9) ) * (20-7)
pts["marker_size"] = size.round(1)

# 등급 색상(불량-보통-우수)
grade_color = {"주의":"#f46d43", "보통":"#fee08b", "우수":"#1a9850"}
pts["color"] = pts["등급"].astype(str).map(grade_color).fillna("#999999")

In [ ]:
# =========================
# 3) 지도 기본
# =========================
center = [36.815, 127.15]  # 천안 중심
m = folium.Map(location=center, zoom_start=12, tiles="cartodbpositron")

# (보조) 얇은 경계선 레이어
fg_border = folium.FeatureGroup(name="읍면동 경계선(보조)", show=False)
folium.GeoJson(
    dong_gdf,
    style_function=lambda x: {"fill": False, "color": "#666", "weight": 1, "opacity":0.6},
    tooltip=folium.GeoJsonTooltip(fields=[EMD_NAME_COL], aliases=["행정동명"])
).add_to(fg_border)
fg_border.add_to(m)

# (핵심) 동 점(예측지수) 레이어
fg_pts = folium.FeatureGroup(name="예측지수(동 중심 점)", show=True)
for _, r in pts.iterrows():
    dong = r["행정동명"]
    score = float(r["청년캠퍼스모빌리티지수"])
    grade = str(r["등급"])
    color = r["color"]
    radius = float(r["marker_size"])

    # 보조지표가 있으면 팝업에 같이 노출
    sub_items = []
    for col_kor in ["업종다양성지수","청년친화업종수","여가 혹은 행정시설물수","교통접근성지수","기숙사수용인원"]:
        if col_kor in pts.columns and pd.notna(r[col_kor]):
            sub_items.append(f"{col_kor}: {r[col_kor]}")
    extra = "<br>".join(sub_items)

    html = f"""
    <div style='font-size:13px;'>
      <b>{dong}</b><br>
      지수: <b>{score:.3f}</b> / 등급: <b>{grade}</b><br>
      {extra}
    </div>
    """
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=radius,
        color=color, fill=True, fill_opacity=0.75, weight=1,
        popup=folium.Popup(html, max_width=280),
        tooltip=f"{dong} · {score:.3f} ({grade})"
    ).add_to(fg_pts)
fg_pts.add_to(m)

# =========================
# (NEW) 연속형 Choropleth 레이어
# =========================
choro_base = dong_gdf.to_crs(4326).copy()
choro_base["key"] = choro_base[EMD_NAME_COL].astype(str)

data_df = df_scores[["행정동명", "청년캠퍼스모빌리티지수"]].copy()
data_df["key"] = data_df["행정동명"].astype(str)

folium.Choropleth(
    geo_data=choro_base,
    data=data_df,
    columns=["key", "청년캠퍼스모빌리티지수"],
    key_on="feature.properties.key",
    fill_color="YlOrRd",
    fill_opacity=0.5,
    line_opacity=0.1,
    legend_name="청년캠퍼스모빌리티지수 (연속)",
    show=False,  # 기본은 꺼두고 필요할 때 켜기
    name="연속 채색(참고)"
).add_to(m)

# =========================
# (NEW) 천안시 전체 외곽선 추가
# =========================
# 기존 읍면동 경계를 하나로 합쳐(dissolve) 천안시 전체 외곽선을 만듭니다.
cheonan_border = dong_gdf.dissolve()

# 두꺼운 외곽선을 별도의 GeoJson 레이어로 추가합니다.
folium.GeoJson(
    cheonan_border,
    style_function=lambda x: {
        "fill": False,          # 안쪽은 채우지 않음
        "color": "#111111",     # 선 색상 (검정색에 가깝게)
        "weight": 4             # 선 두께 (기존보다 두껍게)
    },
    name="천안시 외곽선"
).add_to(m)

In [ ]:
# =========================
# 4) 보조 레이어들
# =========================
# 4-1) 캠퍼스
if "df_geo_ok" in globals() and not df_geo_ok.empty:
    fg_uni = folium.FeatureGroup(name="캠퍼스", show=True)
    for _, u in df_geo_ok.dropna(subset=["lat","lon"]).iterrows():
        nm = str(u.get("학교_std") or u.get("name") or "캠퍼스")
        folium.Marker(
            [float(u["lat"]), float(u["lon"])],
            icon=folium.Icon(color="blue", icon="university", prefix="fa"),
            tooltip=nm,
            popup=nm
        ).add_to(fg_uni)
    fg_uni.add_to(m)

# 4-2) 상권 점포 (최대 1000개 샘플)
if "consume_cheonan" in globals() and not consume_cheonan.empty:
    shops = consume_cheonan.copy()
    for c in ["위도","경도"]:
        if c in shops.columns:
            shops[c] = pd.to_numeric(shops[c], errors="coerce")
    shops = shops.dropna(subset=["위도","경도"])
    if len(shops) > 1000:
        shops = shops.sample(1000, random_state=42)
    fg_shop = folium.FeatureGroup(name=f"상권 점포(표본 {len(shops)}개)", show=False)
    for _, s in shops.iterrows():
        folium.CircleMarker(
            [s["위도"], s["경도"]],
            radius=2.5, color="#ff8c00", fill=True, fill_opacity=0.6,
            tooltip=str(s.get("상호명","점포"))
        ).add_to(fg_shop)
    fg_shop.add_to(m)

# 4-3) 입주기업
if 'company_geocoded' in globals() and not company_geocoded.empty:
    fg_comp = folium.FeatureGroup(name=f"입주기업 ({len(company_geocoded)}개)", show=False)

    for _, row in company_geocoded.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=4,
            color='#7e57c2', # 보라색
            fill=True,
            fill_opacity=0.7,
            tooltip=f"{row['회사명']} ({row['단지명']})"
        ).add_to(fg_comp)

    fg_comp.add_to(m)

In [ ]:
# =========================
# 5) 상단 요약 박스 + 등급 범례
# =========================
mean_score = pts["청년캠퍼스모빌리티지수"].mean()
top5 = pts.sort_values("청년캠퍼스모빌리티지수", ascending=False).head(5)[["행정동명","청년캠퍼스모빌리티지수"]]
top_html = "<br>".join([f"{i+1}. {r['행정동명']} ({r['청년캠퍼스모빌리티지수']:.2f})" for i, r in top5.reset_index(drop=True).iterrows()])

summary_html = f"""
<div style="position: fixed; top: 10px; left: 10px; z-index: 9999;
            background: rgba(255,255,255,0.95); padding: 10px 12px;
            border: 1px solid #ccc; border-radius: 8px; font-size: 12px;">
  <b>천안시 예측지수 요약</b><br>
  평균 지수: <b>{mean_score:.3f}</b><br>
  상위 TOP5<br>{top_html}
</div>
"""
legend_html = """
<div style="position: fixed; top: 450px; right: 10px; z-index: 9999;
             background: rgba(255,255,255,0.95); padding: 10px 12px;
             border: 1px solid #ccc; border-radius: 8px; font-size: 12px;">
  <b>등급(색상)</b><br>
  <div><span style="display:inline-block;width:12px;height:12px;background:#f46d43;border:1px solid #999;margin-right:6px"></span>주의</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:#fee08b;border:1px solid #999;margin-right:6px"></span>보통</div>
  <div><span style="display:inline-block;width:12px;height:12px;background:#1a9850;border:1px solid #999;margin-right:6px"></span>우수</div>
  <div style="margin-top:6px;color:#777;">※ 원 크기 = 지수 크기</div>
</div>
"""
m.get_root().html.add_child(folium.Element(summary_html))
m.get_root().html.add_child(folium.Element(legend_html))

In [ ]:
# =========================
# 6) 저장
# =========================
folium.LayerControl(collapsed=False).add_to(m)
out_name = "천안_예측지도_동점_멀티레이어.html"
m.save(out_name)
print("✅ 저장:", out_name)

✅ 저장: 천안_예측지도_동점_멀티레이어.html


In [ ]:
# 생성된 HTML 파일명
file_name = '천안_예측지도_동점_멀티레이어.html'

# 파일을 다운로드합니다.
files.download(file_name)

print(f"'{file_name}' 파일 다운로드가 시작되었습니다.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'천안_예측지도_동점_멀티레이어.html' 파일 다운로드가 시작되었습니다.
